In [1]:
# CORE LIBRARIES
import os
import re
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as  ns

# NLP LIBRARIES
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# SKLEARN
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## KERAS / TENSORFLOW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional, Dense, Dropout , Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# UTILITIES
import joblib

# CONFIGURATION
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
warnings.filterwarnings("ignore")


2025-12-25 18:03:26.791499: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-25 18:03:26.820228: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-25 18:03:26.972148: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-25 18:03:26.972186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-25 18:03:26.973167: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Dataset Loading and Overview

# Load all datasets
df1 = pd.read_excel("CyberBulling_Dataset_Bangla.xlsx", engine='openpyxl')
df2 = pd.read_excel("cyberbulling bangla dataset.xlsx", engine='openpyxl')
df3 = pd.read_csv("cyberbullying_tweets.csv", encoding='utf-8', encoding_errors='ignore')
df4 = pd.read_csv("romanized_bangla_bullying.csv", encoding='utf-8', on_bad_lines='skip')
# Check columns
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)

Index(['Unnamed: 0', 'Description', 'Label'], dtype='object')
Index(['comment', 'Category', 'Gender', 'comment react number', 'label'], dtype='object')
Index(['tweet_text', 'cyberbullying_type'], dtype='object')
Index(['text', 'label'], dtype='object')


In [3]:
# Explore unique label names
print("Dataset 1 labels:", df1['Label'].unique())
print("Dataset 2 labels:", df2['label'].unique())
print("Dataset 3 labels:", df3['cyberbullying_type'].unique())
print("Dataset 4 labels:", df4['label'].unique())

Dataset 1 labels: ['sexual' 'Threat' 'Political' 'troll' 'Neutral']
Dataset 2 labels: ['religious' 'not bully' 'troll' 'sexual' 'threat']
Dataset 3 labels: ['not_cyberbullying' 'gender' 'religion' 'other_cyberbullying' 'age'
 'ethnicity']
Dataset 4 labels: ['not bullying' 'bullying']


In [4]:
# Label Normalization Function
def normalize_label(label):
    label = str(label).lower().strip()
    if label in ['neutral', 'not bully', 'not_cyberbullying', 'not bullying']:
        return 'Approved'        # Not Bullying
    else:
        return 'Not Approved'    # Bullying

# Apply normalization to each dataset
df1['final_label'] = df1['Label'].apply(normalize_label)
df2['final_label'] = df2['label'].apply(normalize_label)
df3['final_label'] = df3['cyberbullying_type'].apply(normalize_label)
df4['final_label'] = df4['label'].apply(normalize_label)   

# Check distribution
print("Dataset 1:\n", df1['final_label'].value_counts(), "\n")
print("Dataset 2:\n", df2['final_label'].value_counts(), "\n")
print("Dataset 3:\n", df3['final_label'].value_counts(), "\n")
print("Dataset 4:\n", df4['final_label'].value_counts(), "\n")   


Dataset 1:
 final_label
Not Approved    4809
Approved        1201
Name: count, dtype: int64 

Dataset 2:
 final_label
Not Approved    28661
Approved        15340
Name: count, dtype: int64 

Dataset 3:
 final_label
Not Approved    39747
Approved         7945
Name: count, dtype: int64 

Dataset 4:
 final_label
Approved        10093
Not Approved     9907
Name: count, dtype: int64 



In [5]:
# Language Filtering

# Bangla / English detector
def is_bangla(text):
    return bool(re.search('[\u0980-\u09FF]', str(text)))

def is_english(text):
    return bool(re.search('[a-zA-Z]', str(text)))

# Apply detection for df1, df2, df3
for df in [df1, df2, df3]:
    df['language'] = df['Description' if 'Description' in df.columns else 
                        'comment' if 'comment' in df.columns else 
                        'tweet_text'].apply(
        lambda x: 'BN' if is_bangla(x) else ('EN' if is_english(x) else 'OTHER')
    )

# Filter only Bangla and English for these datasets
df1 = df1[df1['language'].isin(['BN', 'EN'])]
df2 = df2[df2['language'].isin(['BN', 'EN'])]
df3 = df3[df3['language'].isin(['BN', 'EN'])]

# Manually tag Romanized Bangla dataset 
df4['language'] = 'RN'

print("Language filtering done.")

# Check Unique languages per dataset
print("Unique languages per dataset:")
print("df1:", df1['language'].unique())
print("df2:", df2['language'].unique())
print("df3:", df3['language'].unique())
print("df4:", df4['language'].unique())

Language filtering done.
Unique languages per dataset:
df1: ['BN']
df2: ['BN']
df3: ['EN']
df4: ['RN']


In [6]:
#  Dataset Merging and Column Unification

# Unify column names for each dataset
df1_final = df1[['Description', 'final_label', 'language']].rename(columns={'Description': 'text', 'final_label': 'label'})
df2_final = df2[['comment', 'final_label', 'language']].rename(columns={'comment': 'text', 'final_label': 'label'})
df3_final = df3[['tweet_text', 'final_label', 'language']].rename(columns={'tweet_text': 'text', 'final_label': 'label'})
df4_final = df4[['text', 'final_label', 'language']].rename(columns={'final_label': 'label'})   # 🔹 Romanized Bangla

# Merge all datasets together
df = pd.concat([df1_final, df2_final, df3_final, df4_final], ignore_index=True)

print(" Final merged dataset shape:", df.shape)

# Preview merged data
df.head()

 Final merged dataset shape: (117677, 3)


,text,label,language
0,ওই হালার পুত এখন কি মদ খাওয়ার সময় রাতের বেলা...,Not Approved,BN
1,আপনার জন্ম প্রক্রিয়ার সময় আপনার মায়ের ভিতর কি ...,Not Approved,BN
2,ধজভংগ দের আর ভায়াগ্রা লাগবো না। ধংস হোক এই সব ...,Not Approved,BN
3,বোকাচোদা একটা।,Not Approved,BN
4,তোর দেশে ফেরার অপেক্ষায় রইলাম। জেলে একটা কামরা...,Not Approved,BN


In [7]:
# Basic info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117677 entries, 0 to 117676
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      117677 non-null  object
 1   label     117677 non-null  object
 2   language  117677 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB
None


In [8]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
text        0
label       0
language    0
dtype: int64


In [9]:
# Duplicate check
duplicates = df.duplicated(subset='text').sum()
print("Duplicate rows:", duplicates)

Duplicate rows: 26303


In [10]:
# Remove duplicates 
df.drop_duplicates(subset='text', inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
# label distribution
print("\nLabel distribution:\n", df['label'].value_counts())


Label distribution:
 label
Not Approved    83100
Approved        34577
Name: count, dtype: int64


In [11]:
# language distribution
print("\nLanguage distribution:\n", df['language'].value_counts())


Language distribution:
 language
BN    50011
EN    47666
RN    20000
Name: count, dtype: int64


In [12]:
# Empty text check
empty_rows = df[df['text'].str.strip() == ""]
print("\nEmpty text rows:", len(empty_rows))


Empty text rows: 0


In [13]:
# check dataset shape
df.shape

(117677, 3)

In [14]:
# TEXT PREPROCESSING 


# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Stopwords

# English stopwords
stop_en = set(stopwords.words('english'))

# Bangla stopwords
try:
    stop_bn = set(stopwords.words('bengali'))
except:
    stop_bn = set()


# Romanized Bangla stopwords (dataset-specific)
stop_rn = {
    'ami', 'amra', 'tumi', 'tomra', 'se', 'tara', 'amar', 'tomar', 'tar', 'tader',
    'eta', 'ota', 'ekta', 'onek', 'sob', 'kisu', 'kichu', 'ke', 'kar', 'kare',
    'jodi', 'jodiw', 'karon', 'tai', 'ekhono', 'age', 'por', 'ekhon',
    'ar', 'kintu', 'ba', 'ta' , 'tobe', 'tore', 'nay', 'na', 'noy', 'ache', 'chilo', 'thake',
    'jonno', 'sate', 'diyese', 'bolse', 'bole', 'korse', 'korche', 'korse', 'korte',
} 


# Text Cleaning Function
def clean_text(text):
    """Clean text: remove URLs, hashtags, punctuation, normalize spaces."""
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # remove URLs
    text = re.sub(r'@\w+|#\w+', ' ', text)                # remove mentions, hashtags
    text = re.sub(r'[^\u0980-\u09FFA-Za-z\s]', ' ', text) # keep Bangla + English
    text = re.sub(r'\s+', ' ', text).strip()              # normalize spaces
    return text


# Preprocessing Function

stemmer = PorterStemmer()

def preprocess_text(text, lang=None):
    """Tokenize, remove stopwords for English/Bangla/Romanized, stem English."""
    text = clean_text(text)
    tokens = word_tokenize(text)
    lang = (lang or '').upper()

    if lang == 'BN':  # Bangla
        tokens = [t for t in tokens if t not in stop_bn and len(t) > 1]

    elif lang == 'EN':  # English
        tokens = [stemmer.stem(t) for t in tokens if t not in stop_en and len(t) > 2]

    elif lang == 'RN':  # Romanized Bangla
        tokens = [t for t in tokens if t not in stop_rn and len(t) > 1]

    else:  # fallback for unknown language
        tokens = [t for t in tokens if len(t) > 1]

    return ' '.join(tokens)


# Apply Preprocessing
df['clean_text'] = df.apply(lambda r: preprocess_text(r['text'], r['language']), axis=1)

# Drop empty cleaned texts
df = df[df['clean_text'].str.strip() != ''].reset_index(drop=True)

print(" Text preprocessing complete.")
print("Sample cleaned texts:\n")

df.head(10)

 Text preprocessing complete.
Sample cleaned texts:



,text,label,language,clean_text
0,ওই হালার পুত এখন কি মদ খাওয়ার সময় রাতের বেলা...,Not Approved,BN,হালার পুত মদ খাওয়ার সময় রাতের বেলা মদ খাই দি...
1,আপনার জন্ম প্রক্রিয়ার সময় আপনার মায়ের ভিতর কি ...,Not Approved,BN,জন্ম প্রক্রিয়ার সময় মায়ের ভিতর বাবা হুমায়ুন কব...
2,ধজভংগ দের আর ভায়াগ্রা লাগবো না। ধংস হোক এই সব ...,Not Approved,BN,ধজভংগ দের ভায়াগ্রা লাগবো ধংস কুলখানকিরা
3,বোকাচোদা একটা।,Not Approved,BN,বোকাচোদা একটা
4,তোর দেশে ফেরার অপেক্ষায় রইলাম। জেলে একটা কামরা...,Not Approved,BN,তোর দেশে ফেরার অপেক্ষায় রইলাম জেলে একটা কামরা ...
5,বাংলাদেশ ফেসবুক ইজ বিনোদন,Not Approved,BN,বাংলাদেশ ফেসবুক ইজ বিনোদন
6,ক্ষমা করার মালিক উপরওয়ালা। উনি ক্ষমা করে দিলে ...,Not Approved,BN,ক্ষমা মালিক উপরওয়ালা ক্ষমা দিলে মানুষের কিইবা ...
7,আপনি ক্ষমা চেয়েছেন। এটাই বড় কথা।ক্ষমা করার মাল...,Not Approved,BN,ক্ষমা চেয়েছেন বড় কথা ক্ষমা মালিক আল্লাহ একজন ম...
8,নিজের রিভিউ বারানোর জন্যই জতো সব নাটক? ফালতু.....,Not Approved,BN,রিভিউ বারানোর জন্যই জতো নাটক ফালতু
9,আপনাদের সকলের কাছে একটা বিষয়ে অনুরোধ-দয়াকরে এই...,Not Approved,BN,আপনাদের সকলের একটা বিষয়ে অনুরোধ দয়াকরে মহিলা ও...


In [ ]:
# Save DataFrame to CSV
df.to_csv('lang_merge.csv', index=False)

In [16]:
# Encode labels (Approved=0, Not Approved=1)
le = LabelEncoder()
df['encoded_label'] = le.fit_transform(df['label'])

# Show mapping
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# Sanity check: balanced labels
print("\nLabel counts:\n", df['encoded_label'].value_counts())

Label mapping: {'Approved': 0, 'Not Approved': 1}

Label counts:
 encoded_label
1    82713
0    34268
Name: count, dtype: int64


In [17]:
# Train-Test Split

X = df['clean_text']
y = df['encoded_label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=SEED,
    stratify=y
)

print(f"Training samples: {len(X_train)} | Testing samples: {len(X_test)}")

Training samples: 93584 | Testing samples: 23397


In [18]:
# Tokenizer + Padding + Embedding Preparation

# Tokenizer setup
MAX_VOCAB = 30000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

print("Tokenizer and padding completed successfully.")
print(f"Training data shape: {X_train_pad.shape}")
print(f"Testing data shape:  {X_test_pad.shape}")


Tokenizer and padding completed successfully.
Training data shape: (93584, 100)
Testing data shape:  (23397, 100)


In [20]:
# ===============================
# Class Weight (Imbalance Handling)
# Approved = 0 (minority)
# Not Approved = 1 (majority)
# ===============================
approved_count = 34577
not_approved_count = 83100
total_samples = approved_count + not_approved_count

class_weight = {
    0: total_samples / (2 * approved_count),
    1: total_samples / (2 * not_approved_count)
}

print("Class Weights:", class_weight)

Class Weights: {0: 1.7016658472394945, 1: 0.7080445246690734}


In [21]:
# ===============================
# Early Stopping
# ===============================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
# ===============================
# Simple RNN Model
# ===============================
rnn_model = Sequential([
    Embedding(
        input_dim=MAX_VOCAB,
        output_dim=128,
        input_length=MAX_LEN
    ),
    SimpleRNN(
        128,
        dropout=0.3,
        recurrent_dropout=0.2
    ),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

rnn_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'AUC']
)

rnn_model.summary()

# ===============================
# Training
# ===============================
history_rnn = rnn_model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop],
    class_weight=class_weight,
    verbose=1
)

# ===============================
# Prediction
# ===============================
y_pred_prob = rnn_model.predict(X_test_pad)
THRESHOLD = 0.35
y_pred = (y_pred_prob > THRESHOLD).astype("int32")

# ===============================
# Metrics
# ===============================
train_acc = history_rnn.history['accuracy'][-1]
val_acc   = history_rnn.history['val_accuracy'][-1]
test_acc  = accuracy_score(y_test, y_pred)
acc_diff  = abs(train_acc - test_acc)

print("\n===== Simple RNN Performance =====")
print(f"Training Accuracy   : {train_acc:.4f}")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Test Accuracy       : {test_acc:.4f}")
print(f"Accuracy Difference : {acc_diff:.4f}")

approved_recall = classification_report(
    y_test, y_pred, output_dict=True
)['0']['recall']

if approved_recall < 0.30:
    print("Model Status: CLASS COLLAPSE")
elif acc_diff <= 0.05:
    print("Model Status: GOOD FIT")
elif train_acc > test_acc + 0.10:
    print("Model Status: OVERFITTING")
else:
    print("Model Status: NEEDS TUNING")

print("\nClassification Report (Simple RNN):\n")
print(classification_report(y_test, y_pred, target_names=['Approved', 'Not Approved']))


In [22]:

# LSTM Model

model = Sequential([
    Embedding(
        input_dim=MAX_VOCAB,
        output_dim=128,
        input_length=MAX_LEN
    ),
    LSTM(
        128,
        dropout=0.3,
        recurrent_dropout=0.2
    ),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# ===============================
# Compile (AUC added)
# ===============================
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'AUC']
)

model.summary()

# ===============================
# Training
# ===============================
history = model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop],
    class_weight=class_weight,
    verbose=1
)

# ===============================
# Prediction (THRESHOLD FIXED)
# ===============================
y_pred_prob = model.predict(X_test_pad)

THRESHOLD = 0.35   # ⭐ FIXED
y_pred = (y_pred_prob > THRESHOLD).astype("int32")

# ===============================
# Accuracy Metrics
# ===============================
train_acc = history.history['accuracy'][-1]
val_acc   = history.history['val_accuracy'][-1]
test_acc  = accuracy_score(y_test, y_pred)
acc_diff  = abs(train_acc - test_acc)

# ===============================
# Performance Summary
# ===============================
print("\n==============================")
print("Model Performance Summary")
print("==============================")
print(f"Training Accuracy   : {train_acc:.4f}")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Test Accuracy       : {test_acc:.4f}")
print(f"Accuracy Difference : {acc_diff:.4f}")

# ===============================
# Correct Model Diagnosis
# ===============================
approved_recall = classification_report(
    y_test, y_pred, output_dict=True
)['0']['recall']

if approved_recall < 0.30:
    print("\n Model Status: CLASS COLLAPSE")
    print(" Approved class is not being learned properly")

elif acc_diff <= 0.05:
    print("\nModel Status: GOOD FIT")
    print(" Model generalizes well")

elif train_acc > test_acc + 0.10:
    print("\n Model Status: OVERFITTING")
    print(" Consider more dropout or less units")

else:
    print("\n Model Status: NEEDS TUNING")

# ===============================
# Classification Report
# ===============================
print("\nClassification Report (LSTM):\n")
print(
    classification_report(
        y_test,
        y_pred,
        target_names=['Approved', 'Not Approved']
    )
)

# ===============================
# Confusion Matrix
# ===============================
plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_pred),
    annot=True,
    fmt='d',
    cmap='Blues'
)
plt.title('Confusion Matrix - LSTM')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


2025-12-25 18:14:33.193338: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          3840000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3979905 (15.18 MB)
Trainable params: 3979905 (15.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
585/5

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM

# ===============================
# BiLSTM Model
# ===============================
bilstm_model = Sequential([
    Embedding(
        input_dim=MAX_VOCAB,
        output_dim=128,
        input_length=MAX_LEN
    ),
    Bidirectional(
        LSTM(
            128,
            dropout=0.3,
            recurrent_dropout=0.2
        )
    ),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

bilstm_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'AUC']
)

bilstm_model.summary()

# ===============================
# Training
# ===============================
history_bilstm = bilstm_model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop],
    class_weight=class_weight,
    verbose=1
)

# ===============================
# Prediction
# ===============================
y_pred_prob = bilstm_model.predict(X_test_pad)
THRESHOLD = 0.35
y_pred = (y_pred_prob > THRESHOLD).astype("int32")

# ===============================
# Metrics
# ===============================
train_acc = history_bilstm.history['accuracy'][-1]
val_acc   = history_bilstm.history['val_accuracy'][-1]
test_acc  = accuracy_score(y_test, y_pred)
acc_diff  = abs(train_acc - test_acc)

print("\n===== BiLSTM Performance =====")
print(f"Training Accuracy   : {train_acc:.4f}")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Test Accuracy       : {test_acc:.4f}")
print(f"Accuracy Difference : {acc_diff:.4f}")

approved_recall = classification_report(
    y_test, y_pred, output_dict=True
)['0']['recall']

if approved_recall < 0.30:
    print("Model Status: CLASS COLLAPSE")
elif acc_diff <= 0.05:
    print("Model Status: GOOD FIT")
elif train_acc > test_acc + 0.10:
    print("Model Status: OVERFITTING")
else:
    print("Model Status: NEEDS TUNING")

print("\nClassification Report (BiLSTM):\n")
print(classification_report(y_test, y_pred, target_names=['Approved', 'Not Approved']))


In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

# ===============================
# 1D CNN Model
# ===============================
cnn_model = Sequential([
    Embedding(
        input_dim=MAX_VOCAB,
        output_dim=128,
        input_length=MAX_LEN
    ),
    Conv1D(
        filters=128,
        kernel_size=5,
        activation='relu'
    ),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'AUC']
)

cnn_model.summary()

# ===============================
# Training
# ===============================
history_cnn = cnn_model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop],
    class_weight=class_weight,
    verbose=1
)

# ===============================
# Prediction
# ===============================
y_pred_prob = cnn_model.predict(X_test_pad)
THRESHOLD = 0.35
y_pred = (y_pred_prob > THRESHOLD).astype("int32")

# ===============================
# Metrics
# ===============================
train_acc = history_cnn.history['accuracy'][-1]
val_acc   = history_cnn.history['val_accuracy'][-1]
test_acc  = accuracy_score(y_test, y_pred)
acc_diff  = abs(train_acc - test_acc)

print("\n===== 1D CNN Performance =====")
print(f"Training Accuracy   : {train_acc:.4f}")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Test Accuracy       : {test_acc:.4f}")
print(f"Accuracy Difference : {acc_diff:.4f}")

approved_recall = classification_report(
    y_test, y_pred, output_dict=True
)['0']['recall']

if approved_recall < 0.30:
    print("Model Status: CLASS COLLAPSE")
elif acc_diff <= 0.05:
    print("Model Status: GOOD FIT")
elif train_acc > test_acc + 0.10:
    print("Model Status: OVERFITTING")
else:
    print("Model Status: NEEDS TUNING")

print("\nClassification Report (1D CNN):\n")
print(classification_report(y_test, y_pred, target_names=['Approved', 'Not Approved']))


In [ ]:
# DL Model Accuracy Comparison

model_scores_dl = {
    "Simple RNN": round(test_acc_rnn, 4),
    "LSTM": round(test_acc_lstm, 4),
    "BiLSTM": round(test_acc_bilstm, 4),
    "1D CNN": round(test_acc_cnn, 4),
}

# Convert to DataFrame and sort by Accuracy
df_scores_dl = pd.DataFrame(model_scores_dl.items(), columns=['Model', 'Accuracy'])
df_scores_dl = df_scores_dl.sort_values(by='Accuracy', ascending=False)

# Display results
print("\n Deep Learning Model Comparison:\n")
print(df_scores_dl.to_string(index=False))

# Plot
plt.figure(figsize=(7,5))
sns.barplot(data=df_scores_dl, x='Model', y='Accuracy', palette='mako')
plt.title('\nModel Accuracy Comparison (Deep Learning)', fontsize=13, fontweight='bold')
plt.xticks(rotation=90)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


In [ ]:
# Evaluate training, validation, and testing accuracy of all Deep Learning Models using 5-Fold Cross Validation
 
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf

def evaluate_dl_model_cv(model, X_train, y_train, X_test, y_test, model_name="Model"):
    print(f"\n{'='*15} {model_name} {'='*15}")

    # Setup
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    acc_train_list, acc_val_list = [], []
    prec_train_list, prec_val_list = [], []
    rec_train_list, rec_val_list = [], []
    f1_train_list, f1_val_list = [], []

    # 5-Fold CV (Training + Validation)
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        # Recreate a fresh copy of the trained model (to avoid weight carryover)
        temp_model = tf.keras.models.clone_model(model)
        temp_model.set_weights(model.get_weights())
        temp_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Evaluate on training fold
        y_tr_pred = (temp_model.predict(X_tr, verbose=0) > 0.5).astype(int)
        y_val_pred = (temp_model.predict(X_val, verbose=0) > 0.5).astype(int)

        # Training metrics
        acc_train_list.append(accuracy_score(y_tr, y_tr_pred))
        prec_train_list.append(precision_score(y_tr, y_tr_pred, zero_division=0))
        rec_train_list.append(recall_score(y_tr, y_tr_pred, zero_division=0))
        f1_train_list.append(f1_score(y_tr, y_tr_pred, zero_division=0))

        # Validation metrics
        acc_val_list.append(accuracy_score(y_val, y_val_pred))
        prec_val_list.append(precision_score(y_val, y_val_pred, zero_division=0))
        rec_val_list.append(recall_score(y_val, y_val_pred, zero_division=0))
        f1_val_list.append(f1_score(y_val, y_val_pred, zero_division=0))

    # Mean ± std for training
    acc_train_mean, acc_train_std = np.mean(acc_train_list), np.std(acc_train_list)
    prec_train_mean, rec_train_mean, f1_train_mean = map(np.mean, [prec_train_list, rec_train_list, f1_train_list])

    # Mean for validation
    acc_val_mean, prec_val_mean, rec_val_mean, f1_val_mean = map(np.mean, [acc_val_list, prec_val_list, rec_val_list, f1_val_list])

    # Test performance using already trained model
    y_pred_test = (model.predict(X_test, verbose=0) > 0.5).astype(int)
    acc_test = accuracy_score(y_test, y_pred_test)
    prec_test = precision_score(y_test, y_pred_test, zero_division=0)
    rec_test = recall_score(y_test, y_pred_test, zero_division=0)
    f1_test = f1_score(y_test, y_pred_test, zero_division=0)

    # Print Results 
    print("\nTraining Performance (5-Fold CV):")
    print(f"Accuracy:  {acc_train_mean:.4f} ± {acc_train_std:.4f}")
    print(f"Precision: {prec_train_mean:.4f}")
    print(f"Recall:    {rec_train_mean:.4f}")
    print(f"F1-score:  {f1_train_mean:.4f}")

    print("\nValidation Performance (5-Fold CV):")
    print(f"Accuracy:  {acc_val_mean:.4f}")
    print(f"Precision: {prec_val_mean:.4f}")
    print(f"Recall:    {rec_val_mean:.4f}")
    print(f"F1-score:  {f1_val_mean:.4f}")

    print("\nTesting Performance:")
    print(f"Accuracy:  {acc_test:.4f}")
    print(f"Precision: {prec_test:.4f}")
    print(f"Recall:    {rec_test:.4f}")
    print(f"F1-score:  {f1_test:.4f}")

    # Generalization Check
    acc_gap = acc_train_mean - acc_test
    f1_gap = f1_train_mean - f1_test

    if acc_gap > 0.05 or f1_gap > 0.05:
        print("\n Potential Overfitting: Training metrics significantly higher than test metrics.")
    elif acc_train_mean < 0.75 and acc_test < 0.75:
        print("\n Potential Underfitting: Both training and test accuracies are low.")
    else:
        print("\n The model seems to generalize well with balanced training and testing performance.")


# Run on all trained DL models
evaluate_dl_model_cv(rnn_model, X_train_pad, y_train.values, X_test_pad, y_test.values, "Simple RNN")
evaluate_dl_model_cv(lstm_model, X_train_pad, y_train.values, X_test_pad, y_test.values, "LSTM")
evaluate_dl_model_cv(bilstm_model, X_train_pad, y_train.values, X_test_pad, y_test.values, "BiLSTM")
evaluate_dl_model_cv(cnn_model, X_train_pad, y_train.values, X_test_pad, y_test.values, "1D CNN")


In [ ]:
# ROC–AUC Visualization - DL Models

from sklearn.metrics import roc_curve, auc

# Probabilities from trained models
y_prob_rnn = rnn_model.predict(X_test_pad)
y_prob_lstm = lstm_model.predict(X_test_pad)
y_prob_bilstm = bilstm_model.predict(X_test_pad)
y_prob_cnn = cnn_model.predict(X_test_pad)

# Compute ROC curve and AUC
fpr_rnn, tpr_rnn, _ = roc_curve(y_test, y_prob_rnn)
fpr_lstm, tpr_lstm, _ = roc_curve(y_test, y_prob_lstm)
fpr_bilstm, tpr_bilstm, _ = roc_curve(y_test, y_prob_bilstm)
fpr_cnn, tpr_cnn, _ = roc_curve(y_test, y_prob_cnn)

auc_rnn = auc(fpr_rnn, tpr_rnn)
auc_lstm = auc(fpr_lstm, tpr_lstm)
auc_bilstm = auc(fpr_bilstm, tpr_bilstm)
auc_cnn = auc(fpr_cnn, tpr_cnn)

# Plot ROC Curves
plt.figure(figsize=(8,6))
plt.plot(fpr_rnn, tpr_rnn, label=f"Simple RNN (AUC = {auc_rnn:.3f})")
plt.plot(fpr_lstm, tpr_lstm, label=f"LSTM (AUC = {auc_lstm:.3f})")
plt.plot(fpr_bilstm, tpr_bilstm, label=f"BiLSTM (AUC = {auc_bilstm:.3f})")
plt.plot(fpr_cnn, tpr_cnn, label=f"1D CNN (AUC = {auc_cnn:.3f})")
plt.plot([0, 1], [0, 1], 'k--', label="Chance (AUC = 0.5)")

plt.title("ROC Curves – Deep Learning Models")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()



BiLSTM and 1D CNN show the strongest performance (AUC ≈ 0.935), clearly separating classes.  
Simple RNN performs near random (AUC ≈ 0.52), while LSTM underperforms (AUC ≈ 0.50).  
Overall, **BiLSTM and 1D CNN** are the best deep models for this task.
